In [202]:
import pandas as pd

prolongations = pd.read_csv('prolongations.csv')
financial_data = pd.read_csv('financial_data.csv')

financial_data.replace('\xa0', '', regex=True, inplace=True)
financial_data.replace(',', '.', regex=True, inplace=True)

In [203]:
managers = prolongations["AM"].unique()

In [204]:
all_months = ['ноябрь 2022', 'декабрь 2022', 'январь 2023', 'февраль 2023',
              'март 2023', 'апрель 2023', 'май 2023', 'июнь 2023', 'июль 2023',
              'август 2023', 'сентябрь 2023', 'октябрь 2023', 'ноябрь 2023',
              'декабрь 2023', 'январь 2024', 'февраль 2024']

months_to_calc_prolongation = ['январь 2023', 'февраль 2023', 'март 2023', 'апрель 2023',
                  'май 2023', 'июнь 2023', 'июль 2023', 'август 2023',
                  'сентябрь 2023', 'октябрь 2023', 'ноябрь 2023', 'декабрь 2023']

In [205]:
# для начала посчитаем коэффициенты пролонгации для каждого менеджера:
# 1) за каждый месяц (2023 год)
# 2) за год (2023 год)

In [206]:
def capitalize_first_letter(string):
    return string[0].upper() + string[1:]

def replace_zeros_with_previous(df):
    for index, row in df.iterrows():
        previous_value = None
        for column in df.columns[1:]:
            if row[column] != 0:
                previous_value = row[column]
            elif row[column] == 0 and previous_value is not None:
                df.at[index, column] = previous_value
    
    return df

def remove_rows_with_zeros(df):
    return df[~df.apply(lambda row: (row == 0).any(), axis=1)]

In [207]:
import numpy as np

for manager in managers:
    for prolongation_month in months_to_calc_prolongation:
        # получаем все проекты, которые были завершены в предыдущих месяцах
        prolongation_month_index = all_months.index(prolongation_month)
        month0_index = all_months.index(prolongation_month) - 2
        month1_index = all_months.index(prolongation_month) - 1
        month0 = all_months[month0_index]
        month1 = all_months[month1_index]

        manager_completed_projects = prolongations[prolongations['AM'] == manager]
        
        month0_completed_projects = manager_completed_projects[manager_completed_projects['month'] == month0]
        month1_completed_projects = manager_completed_projects[manager_completed_projects['month'] == month1]
        
        k2_up_sum = 0
        k2_down_sum = 0
       
        # СЧИТАЕМ ПРОЕКТЫ, ЗАВЕРШЕННЫЕ В МЕСЯЦЕ 0 (КОЭФФИЦИЕНТ K2)
        ids = month0_completed_projects['id'].unique()
        projects_financial_data = financial_data[financial_data['id'].isin(ids)].copy()
        
        for id, project in projects_financial_data.iterrows():
            for check_month in range(0, prolongation_month_index + 1):
                if project[capitalize_first_letter(all_months[check_month])] == "стоп" or project[capitalize_first_letter(all_months[check_month])] == "end":
                    projects_financial_data.drop(id, inplace=True)
        
        required_months = all_months[0: prolongation_month_index + 1]
        required_months = [capitalize_first_letter(month) for month in required_months]
        required_months = ['id'] + required_months

        projects_financial_data = projects_financial_data[required_months]

        projects_financial_data = projects_financial_data.replace("в ноль", 0)
        projects_financial_data = projects_financial_data.replace("NaN", np.nan)
        projects_financial_data = projects_financial_data.apply(pd.to_numeric, errors='coerce')

        projects_financial_data_grouped = projects_financial_data.groupby('id').sum(min_count=1)
        projects_financial_data_grouped = replace_zeros_with_previous(projects_financial_data_grouped)
        projects_financial_data_grouped = remove_rows_with_zeros(projects_financial_data_grouped)
        
        print("Итоговый датафрейм для менеджера", manager, "за prolongation месяц", prolongation_month)
        print(projects_financial_data_grouped)
        
        # k2 up = сумма отгрузки всех проектов пролонгированных во второй месяц после завершения, например сумма отгрузки тех проектов, завершившихся в марте, у которых нет отгрузки в апреле, но есть в мае (за май).
        # k2 down = сумма отгрузки последнего месяца проектов не пролонгированных в первый, например сумма отгрузки проектов, завершившихся в марте, у которых нет отгрузки в апреле (за март)
        # нет отгрузки == NaN
        for index, row in projects_financial_data_grouped.iterrows():
            if pd.isna(row[capitalize_first_letter(month1)]) and row[capitalize_first_letter(prolongation_month)] > 0 and row[capitalize_first_letter(month0)] > 0:
                print(f"Прибавляем {row[capitalize_first_letter(prolongation_month)]} к k2_up_sum")
                k2_up_sum += row[capitalize_first_letter(prolongation_month)]
            if pd.isna(row[capitalize_first_letter(month1)]) and row[capitalize_first_letter(month0)] > 0:
                print(f"Прибавляем {row[capitalize_first_letter(month0)]} к k2_down_sum")
                k2_down_sum += row[capitalize_first_letter(month0)]
                
        if k2_down_sum > 0:
            k2 = k2_up_sum / k2_down_sum
        else:
            k2 = 0
            
        print(f"Месяц {prolongation_month} менеджера {manager}: k2 = {k2}")

Итоговый датафрейм для менеджера Васильев Артем Александрович за prolongation месяц январь 2023
     Ноябрь 2022  Декабрь 2022  Январь 2023
id                                         
42       36220.0           NaN          NaN
369      42600.0        7450.0          NaN
592     104642.0           NaN          NaN
600     281417.0      175100.0     267220.0
633      40200.0       40200.0      40200.0
665      10000.0           NaN          NaN
666     110000.0           NaN          NaN
Прибавляем 36220.0 к k2_down_sum
Прибавляем 104642.0 к k2_down_sum
Прибавляем 10000.0 к k2_down_sum
Прибавляем 110000.0 к k2_down_sum
Месяц январь 2023 менеджера Васильев Артем Александрович: k2 = 0.0
Итоговый датафрейм для менеджера Васильев Артем Александрович за prolongation месяц февраль 2023
     Ноябрь 2022  Декабрь 2022  Январь 2023  Февраль 2023
id                                                       
31      55100.00      55100.00          NaN      44775.00
46      37939.50      37939.50      